In [30]:
import torch
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split

from torch.utils.data import DataLoader, Dataset, TensorDataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import roc_auc_score, accuracy_score
import datetime
import itertools
import warnings
warnings.filterwarnings('ignore')

In [31]:
class Dnn(nn.Module):
    def __init__(self, hidden_units, dropout=0.):
        super(Dnn, self).__init__()
        self.dnn_network = nn.ModuleList([nn.Linear(layer[0], layer[1]) for layer in list(zip(hidden_units[:-1], hidden_units[1:]))])
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        for linear in self.dnn_network:
            x = linear(x)
            x = F.relu(x)
        x = self.dropout(x)
        return x
        
class Attention_layer(nn.Module):
    def __init__(self, att_units):
        """
        att_units:[embed_dim, att_vector]
        """
        super(Attention_layer, self).__init__()
        self.att_w = nn.Linear(att_units[0], att_units[1])
        self.att_dense = nn.Linear(att_units[1], 1)
    def forward(self, bi_interation):
        a = self.att_w(bi_interation)  #bi_iteation(None, field_num*(field_num-1)/2, embed_dim)
        a = F.relu(a)
        att_scores = self.att_dense(a) #（None，field_num*(field_num-1)/2, 1)
        att_weight = F.softmax(att_scores, dim=1) #(None, field_num*(field_num-1)/2, 1)
        att_out = torch.sum(att_weight * bi_interation, dim=1)
        return att_out
        
class AFM(nn.Module):
    def __init__(self, feature_columns, mode, hidden_units, att_vector=8, dropout=0.5, useDNN=False):
        """
        feature_columns:特征信息
        mode:三种模式,'max':max pooling, 'avg':average pooling, 'att':attention
        att_vector:注意力网络的隐藏层单元个数
        hidden_units:DNN网络的隐藏单元个数
        dropout:Dropout比率
        useDNN:默认不使用DNN网络
        """
        super(AFM, self).__init__()
        self.dense_feature_cols, self.sparse_feature_cols = feature_columns
        self.mode = mode
        self.useDNN = useDNN
        
        #embedding
        self.embed_layers = nn.ModuleDict({
            'embed_'+str(i):nn.Embedding(num_embeddings=feat['feat_num'], embedding_dim=feat['embed_dim']) for i, feat in enumerate(self.sparse_feature_cols)
        })
        
        #如果这里是注意力机制的话，这里需要加一个注意力网络
        if self.mode == 'att':
            self.attention = Attention_layer([self.sparse_feature_cols[0]['embed_dim'], att_vector])
        
        #如果使用DNN的话，这里需要初始化DNN网络
        if self.useDNN:
            self.fea_num = len(self.dense_feature_cols) + self.sparse_feature_cols[0]['embed_dim']
            hidden_units.insert(0, self.fea_num)
            
            self.bn = nn.BatchNorm1d(self.fea_num)
            self.dnn_network = Dnn(hidden_units, dropout)
            self.nn_final_linear = nn.Linear(hidden_units[-1], 1)
        else:
            self.fea_num = len(self.dense_feature_cols) + self.sparse_feature_cols[0]['embed_dim']
            self.nn_final_linear = nn.Linear(self.fea_num, 1)
    def forward(self, x):
        dense_inputs, sparse_inputs = x[:, :len(self.dense_feature_cols)], x[:, len(self.dense_feature_cols):]
        sparse_inputs = sparse_inputs.long()
        sparse_embeds = [self.embed_layers['embed_'+str(i)](sparse_inputs[:, i]) for i in range(sparse_inputs.shape[1])]
        sparse_embeds = torch.stack(sparse_embeds)
        sparse_embeds = sparse_embeds.permute((1, 0, 2))
        #这里得到embedding向量之后sparse_embeds(None, field_num, embed_dim)
        #下面进行两两交叉，注意这时不能加和了，这里两两交叉的结果要进入attention
        #两两交叉embedding之后的结果是一个(None, field_num*(field_num-1)/2, embed_dim)
        #这里实现的时候采用组合这个技巧
        #比如field_num有4个的话，那么组合embedding就是[0,1],[0,2],[0,3],[1,2],[1,3],[2,3]位置的embedding乘积操作
        first = []
        second = []
        for f, s in itertools.combinations(range(sparse_embeds.shape[1]), 2):
            first.append(f)
            second.append(s)
        #取出first位置的embedding，假设field是4的话，就是[0,0,0,1,1,2]位置的embedding
        p = sparse_embeds[:, first, :] #(None, field_num*(field_num-1)/2, embed_dim)
        q = sparse_embeds[:, second, :] #(None, field_num*(field_num-1)/2, embed_dim)
        bi_interaction = p * q
        
        if self.mode == 'max':
            att_out = torch.sum(bi_interaction, dim=1) #(None, embed_dim)
        elif self.mode == 'avg':
            att_out = torch.mean(bi_interaction, dim=1) #(None, embed_dim)
        else:
            #注意力网络
            att_out = self.attention(bi_interaction)
        
        #把离散特征和连续特征进行拼接
        x = torch.cat([att_out, dense_inputs], dim=-1)
        
        if not self.useDNN:
            outputs = F.sigmoid(self.nn_final_linear(x))
        else:
            #BatchNormalization
            x = self.bn(x)
            #deep
            dnn_outputs = self.nn_final_linear(self.dnn_network(x))
            outputs = F.sigmoid(dnn_outputs)
        
        return outputs

In [32]:
data = pd.read_csv('./data/Criteo_sample.txt')
train_set, test_set = train_test_split(data, test_size=0.2, random_state=2024)

train_label = train_set['label']
del train_set['label']
test_label = test_set['label']
del test_set['label']
data_df = pd.concat((train_set, test_set))


sparse_feas = [col for col in data_df.columns if col[0] == 'C']
dense_feas = [col for col in data_df.columns if col[0] == 'I']

data_df[sparse_feas] = data_df[sparse_feas].fillna('-1')
data_df[dense_feas] = data_df[dense_feas].fillna(0)

def sparseFeature(feat, feat_num, embed_dim=4):
    return {'feat':feat, 'feat_num':feat_num, 'embed_dim':embed_dim}
def denseFeature(feat):
    return {'feat':feat}
embed_dim = 8
feature_columns = [[denseFeature(feat) for feat in dense_feas]] + [[sparseFeature(feat, len(data_df[feat].unique()), embed_dim=embed_dim) for feat in sparse_feas]]

for feat in sparse_feas:
    le = LabelEncoder()
    data_df[feat] = le.fit_transform(data_df[feat])

mms = MinMaxScaler()
data_df[dense_feas] = mms.fit_transform(data_df[dense_feas])

train = data_df[:train_set.shape[0]]
test = data_df[train_set.shape[0]:]

train['label'] = train_label
test['label'] = test_label

train_set, val_set = train_test_split(train, test_size=0.1, random_state=2020)

train_set.reset_index(drop=True, inplace=True)
val_set.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)
print(feature_columns)
print(train_set.shape, val_set.shape, test.shape)

[[{'feat': 'I1'}, {'feat': 'I2'}, {'feat': 'I3'}, {'feat': 'I4'}, {'feat': 'I5'}, {'feat': 'I6'}, {'feat': 'I7'}, {'feat': 'I8'}, {'feat': 'I9'}, {'feat': 'I10'}, {'feat': 'I11'}, {'feat': 'I12'}, {'feat': 'I13'}], [{'feat': 'C1', 'feat_num': 79, 'embed_dim': 8}, {'feat': 'C2', 'feat_num': 252, 'embed_dim': 8}, {'feat': 'C3', 'feat_num': 1293, 'embed_dim': 8}, {'feat': 'C4', 'feat_num': 1043, 'embed_dim': 8}, {'feat': 'C5', 'feat_num': 30, 'embed_dim': 8}, {'feat': 'C6', 'feat_num': 7, 'embed_dim': 8}, {'feat': 'C7', 'feat_num': 1164, 'embed_dim': 8}, {'feat': 'C8', 'feat_num': 39, 'embed_dim': 8}, {'feat': 'C9', 'feat_num': 2, 'embed_dim': 8}, {'feat': 'C10', 'feat_num': 908, 'embed_dim': 8}, {'feat': 'C11', 'feat_num': 926, 'embed_dim': 8}, {'feat': 'C12', 'feat_num': 1239, 'embed_dim': 8}, {'feat': 'C13', 'feat_num': 824, 'embed_dim': 8}, {'feat': 'C14', 'feat_num': 20, 'embed_dim': 8}, {'feat': 'C15', 'feat_num': 819, 'embed_dim': 8}, {'feat': 'C16', 'feat_num': 1159, 'embed_dim': 

In [33]:
#建立模型
hidden_units = [128, 64, 32]
dnn_dropout = 0.
model = AFM(feature_columns, 'att', hidden_units, dropout=dnn_dropout, useDNN=True)
model

AFM(
  (embed_layers): ModuleDict(
    (embed_0): Embedding(79, 8)
    (embed_1): Embedding(252, 8)
    (embed_2): Embedding(1293, 8)
    (embed_3): Embedding(1043, 8)
    (embed_4): Embedding(30, 8)
    (embed_5): Embedding(7, 8)
    (embed_6): Embedding(1164, 8)
    (embed_7): Embedding(39, 8)
    (embed_8): Embedding(2, 8)
    (embed_9): Embedding(908, 8)
    (embed_10): Embedding(926, 8)
    (embed_11): Embedding(1239, 8)
    (embed_12): Embedding(824, 8)
    (embed_13): Embedding(20, 8)
    (embed_14): Embedding(819, 8)
    (embed_15): Embedding(1159, 8)
    (embed_16): Embedding(9, 8)
    (embed_17): Embedding(534, 8)
    (embed_18): Embedding(201, 8)
    (embed_19): Embedding(4, 8)
    (embed_20): Embedding(1204, 8)
    (embed_21): Embedding(7, 8)
    (embed_22): Embedding(12, 8)
    (embed_23): Embedding(729, 8)
    (embed_24): Embedding(33, 8)
    (embed_25): Embedding(554, 8)
  )
  (attention): Attention_layer(
    (att_w): Linear(in_features=8, out_features=8, bias=True)
   

In [34]:
trn_x = train_set.drop('label', axis=1).values
trn_y = train_set['label'].values
val_x = val_set.drop('label', axis=1).values
val_y = val_set['label'].values
dl_train_dataset = TensorDataset(torch.tensor(trn_x).float(), torch.tensor(trn_y).float())
dl_val_dataset = TensorDataset(torch.tensor(val_x).float(), torch.tensor(val_y).float())

dl_train = DataLoader(dl_train_dataset, shuffle=True, batch_size=32)
dl_val = DataLoader(dl_val_dataset, shuffle=True, batch_size=32)

In [35]:
def auc(y_pred, y_true):
    pred = y_pred.data
    y = y_true.data
    return roc_auc_score(y, pred)

loss_func = nn.BCELoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)
metric_func = auc
metric_name = 'auc'
epochs = 10
log_step_freq = 10

dfhistory = pd.DataFrame(columns=['epoch', 'loss', metric_name, 'val_loss', 'val_'+metric_name])

print('start_training.........')

print('========'*8)

for epoch in range(1, epochs+1):
    
    # 训练阶段
    model.train()
    loss_sum = 0.0
    metric_sum = 0.0
    step = 1
    

    
    for step, (features, labels) in enumerate(dl_train, 1):
        # 梯度清零
        optimizer.zero_grad()
        labels = labels.view(-1, 1)
        
        # 正向传播
        predictions = model(features);
        loss = loss_func(predictions, labels)
        try:
            metric = metric_func(predictions, labels)
        except ValueError:
            pass
        
        # 反向传播
        loss.backward()
        optimizer.step()
        
        # 打印batch级别日志
        loss_sum += loss.item()
        metric_sum += metric.item()
        if step % log_step_freq == 0:
            print(("[step=%d] loss: %.3f, " + metric_name + ": %.3f") % (step, loss_sum/step, metric_sum/step));
    
    # 验证阶段
    model.eval()
    val_loss_sum = 0.0
    val_metric_sum = 0.0
    val_step = 1
    
    for val_step, (features, labels) in enumerate(dl_val, 1):
        labels = labels.view(-1, 1)
        with torch.no_grad():
            predictions = model(features)
            val_loss = loss_func(predictions, labels)
            try:
                val_metric = metric_func(predictions, labels)
            except ValueError:
                pass
        
        val_loss_sum += val_loss.item()
        val_metric_sum += val_metric.item()
    
    # 记录日志
    info = (epoch, loss_sum/step, metric_sum/step, val_loss_sum/val_step, val_metric_sum/val_step)
    dfhistory.loc[epoch-1] = info
    
    # 打印日志
    print(("\nEPOCH=%d, loss=%.3f, " + metric_name + " = %.3f, val_loss=%.3f, " + "val_" + metric_name + " = %.3f") %info)
    print('\n' + '=========='* 8)
    
print('Finished Training')

start_training.........
[step=10] loss: 0.701, auc: 0.512
[step=20] loss: 0.696, auc: 0.517
[step=30] loss: 0.691, auc: 0.506
[step=40] loss: 0.687, auc: 0.476

EPOCH=1, loss=0.685, auc = 0.479, val_loss=0.676, val_auc = 0.579

[step=10] loss: 0.656, auc: 0.522
[step=20] loss: 0.648, auc: 0.520
[step=30] loss: 0.647, auc: 0.494
[step=40] loss: 0.640, auc: 0.487

EPOCH=2, loss=0.637, auc = 0.485, val_loss=0.607, val_auc = 0.576

[step=10] loss: 0.590, auc: 0.496
[step=20] loss: 0.599, auc: 0.496
[step=30] loss: 0.589, auc: 0.505
[step=40] loss: 0.583, auc: 0.501

EPOCH=3, loss=0.578, auc = 0.502, val_loss=0.547, val_auc = 0.530

[step=10] loss: 0.563, auc: 0.487
[step=20] loss: 0.552, auc: 0.529
[step=30] loss: 0.542, auc: 0.537
[step=40] loss: 0.535, auc: 0.538

EPOCH=4, loss=0.528, auc = 0.550, val_loss=0.520, val_auc = 0.574

[step=10] loss: 0.480, auc: 0.597
[step=20] loss: 0.501, auc: 0.552
[step=30] loss: 0.508, auc: 0.564
[step=40] loss: 0.508, auc: 0.569

EPOCH=5, loss=0.506, au

In [36]:
# 预测
from sklearn.metrics import accuracy_score
test_x = test.drop('label', axis=1).values
test_y = test['label'].values
y_pred_probs = model(torch.tensor(test_x).float())
y_pred = torch.where(y_pred_probs>0.5, torch.ones_like(y_pred_probs), torch.zeros_like(y_pred_probs))

test_auc = roc_auc_score(test_y, y_pred_probs.data.numpy())
test_acc = accuracy_score(test_y, y_pred.data.numpy())
print('test_auc:%.3f test_acc:%.3f'%(test_auc, test_acc))

test_auc:0.666 test_acc:0.765
